In [3]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time 
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
import configparser

In [6]:
# 1. 로그인

In [ ]:
#userId = 
#userPss = 

In [39]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [40]:
driver.set_window_size(1400, 1200)
driver.get("https://www.kaida.co.kr/uat/uia/egovLoginUsr.do")

In [41]:
driver.find_element_by_css_selector('#userId').send_keys(user_id)
driver.find_element_by_css_selector('#userPass').send_keys(user_pass)
driver.execute_script('actionLogin()')

In [42]:
# 2. 사용자 설정통계 페이지 이동

In [43]:
driver.get("https://www.kaida.co.kr/ko/statistics/custom2.do")

In [12]:
# 3. 브랜드, 지역, 구매유형, 연령 데이터 수집

In [13]:
# 브랜드
brands = driver.find_elements_by_css_selector("#td_Brand > .item_wrap > label")
brands = {brand.text: brand.get_attribute("for") for brand in brands}
print(len(brands), brands)

23 {'Audi': 'Brand001', 'BMW': 'Brand002', 'Bentley': 'Brand035', 'Cadillac': 'Brand006', 'Chevrolet': 'Brand057', 'Chrysler': 'Brand003', 'Citroen': 'Brand020', 'Ford': 'Brand005', 'Honda': 'Brand007', 'Jaguar': 'Brand008', 'Lamborghini': 'Brand034', 'Land Rover': 'Brand009', 'Lexus': 'Brand015', 'Lincoln': 'Brand058', 'MINI': 'Brand039', 'Maserati': 'Brand010', 'Mercedes-Benz': 'Brand011', 'Peugeot': 'Brand012', 'Porsche': 'Brand013', 'Rolls-Royce': 'Brand032', 'Toyota': 'Brand044', 'Volkswagen': 'Brand016', 'Volvo': 'Brand017'}


In [14]:
# 지역
areas = driver.find_elements_by_css_selector("#td_City > .item_wrap > label")
areas = {area.text: area.get_attribute("for") for area in areas}
print(len(areas), areas)

17 {'서울': 'City11', '부산': 'City26', '대구': 'City27', '인천': 'City28', '광주': 'City29', '대전': 'City30', '울산': 'City31', '세종': 'City36', '경기': 'City41', '강원': 'City42', '충북': 'City43', '충남': 'City44', '전북': 'City45', '전남': 'City46', '경북': 'City47', '경남': 'City48', '제주': 'City49'}


In [15]:
# 구매유형
selector = '#pageBody > article.body_wrap > div > div.select_filter > table > tbody > tr > td:nth-child(8) > div > label'
sales_types = driver.find_elements_by_css_selector(selector)
sales_types = {sales_type.text: sales_type.get_attribute("for") for sales_type in sales_types}
print(len(sales_types), sales_types)

4 {'개인': 'PurchaseType3', '개인-남자': 'PurchaseType1', '개인-여자': 'PurchaseType2', '법인': 'PurchaseType0'}


In [16]:
# 연령
ages = driver.find_elements_by_css_selector(".td_Age > .item_wrap > label")
ages = {age.text: age.get_attribute("for") for age in ages}
print(len(ages), ages)

7 {'~19': 'Age1', '20~29': 'Age2', '30~39': 'Age3', '40~49': 'Age4', '50~59': 'Age5', '60~69': 'Age6', '70~': 'Age7'}


In [17]:
# 4. 선택

In [18]:
def clear_menu(driver):
    # 지역 체크박스 초기화 
    driver.execute_script("$('#td_City > .item_wrap > label > input').prop('checked', false);")
    # 나이 체크박스 초기화
    driver.execute_script("$('#td_Age > .item_wrap > label > input').prop('checked', false);")
    # tag 초기화
    driver.execute_script("$('.sel_item button').click();");

In [19]:
def select_menu(driver, area, area_id, sales_type_id, age_id):
    
    # 메뉴 초기화
    clear_menu(driver)
    time.sleep(0.5)
    
    # 지역 클릭
    driver.find_element_by_css_selector(f'input#{area_id}').click()
    time.sleep(0.5)
    
    # 상세 지역 클릭 : 전체가 클릭되도록 설정 : 이 메뉴가 안나오는 경우도 있으므로 예외처리
    try:
        driver.find_element_by_css_selector(f'input#all-{area}').click()
        time.sleep(0.5)
    except:
        pass
    
    # 구매유형 클릭
    driver.find_element_by_css_selector(f'input#{sales_type_id}').click()
    time.sleep(0.5)
    
    # 연령대 클릭
    try: 
        driver.find_element_by_css_selector(f'input#{age_id}').click()
        time.sleep(0.5)
    except:
        pass

In [20]:
# 5. 데이터 수집

In [21]:
def make_table_df(driver, area, sales_type, age):
    
    # 컬럼 데이터 수집
    columns = driver.find_elements_by_css_selector("table.re_table > thead > tr > th")
    columns = [column.text for column in columns] + ["area", "sales_type", "age"]

    # 데이터 프레임 만들기
    df = pd.DataFrame(columns=columns)

    # 데이터 프레임 채우기
    rows = driver.find_elements_by_css_selector("table.re_table > tbody > tr")
    for row in rows:
        cells = row.find_elements_by_css_selector('td,th')
        datas = [cell.text for cell in cells] + [area, sales_type, age]     
        df.loc[len(df)] = datas
    
    return df

In [37]:
# 6. 함수 실행
dfs = []

for year in range(2017, 2018): #2016, 2022        
    years = Select(driver.find_element_by_xpath("""//*[@id="searchYear"]"""))
    years.select_by_value(f'{year}')  
    for area, area_id in list(areas.items()):
        for sales_type, sales_type_id in list(sales_types.items())[:3]: #법인은 오류가 나서 뺐습니다 ~.~
            for age, age_id in list(ages.items()):
                print([area, sales_type, age, year], end=" ")

                    # 메뉴 선택
                select_menu(driver, area, area_id, sales_type_id, age_id)

                    # 검색버튼 클릭
                driver.find_element_by_css_selector('.searchBtn').click()
                time.sleep(0.5)

                    # 데이터 수집해서 데이터 프레임으로 만들기
                df = make_table_df(driver, area, sales_type, age)
                dfs.append(df)

                
# 수집된 데이터 프레임 병합
result_df = pd.concat(dfs)

# index 초기화
result_df.reset_index(drop=True, inplace=True)

# csv 파일로 저장
result_df.to_csv("result.csv", index=False)
result_df.to_excel('result_2016.xlsx', index=False)

# 출력
result_df.tail()

['서울', '법인', '60~69', 2017] ['부산', '법인', '60~69', 2017] ['대구', '법인', '60~69', 2017] ['인천', '법인', '60~69', 2017] ['광주', '법인', '60~69', 2017] ['대전', '법인', '60~69', 2017] ['울산', '법인', '60~69', 2017] 

KeyboardInterrupt: 

In [21]:
result_df.to_excel('result_2016.xlsx', index=False)

In [22]:
result_df

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,area,sales_type,age
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5041,Volvo,0,0,0,0,1,0,0,0,0,0,0,0,1,제주,개인-여자,60~69
5042,Jaguar,0,0,0,0,0,1,0,0,0,0,0,0,1,제주,개인-여자,70~
5043,Lexus,0,0,1,0,0,0,1,0,0,0,0,0,2,제주,개인-여자,70~
5044,Mercedes-Benz,0,0,0,1,0,0,0,0,0,1,0,0,2,제주,개인-여자,70~


In [23]:
driver.quit()

In [34]:
df_2021 = pd.read_excel('./result/result_2021.xlsx')
df_2020 = pd.read_excel('./result/result_2020.xlsx')
df_2019 = pd.read_excel('./result/result_2019.xlsx')
df_2018 = pd.read_excel('./result/result_2018.xlsx')
df_2017 = pd.read_excel('./result/result_2017.xlsx')
df_2016 = pd.read_excel('./result/result_2016.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: './result/result_2021.xlsx'

In [35]:
df = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021])

In [31]:
df

NameError: name 'df' is not defined

In [36]:
df.to_csv('kaida.csv', index=False, encoding='utf-8-sig')

In [31]:
df.head()

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,area,sales_type,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016


In [37]:
df.columns = ["Brand", "1월", "2월", "3월", "4월", "5월", "6월", "7월", "8월", "9월", "10월", "11월", "12월", "total", "지역", "유형", "age", "year"]

In [32]:
c


,year
0,2016
1,2016
2,2016
3,2016
4,2016
...,...
33427,2021
33428,2021
33429,2021
33430,2021


In [30]:
df.T

NameError: name 'df' is not defined

In [35]:
df.to_csv('kaida_korean.csv', index=False, encoding='utf-8-sig')

NameError: name 'df' is not defined

In [53]:
d = pd.read_csv('kaida_korean.csv')
d

,Brand,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,total,지역,유형,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33427,Peugeot,2,0,14,27,0,0,0,0,0,0,0,0,43,제주,법인,-,2021
33428,Porsche,3,1,2,5,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
33429,Toyota,1,4,3,3,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
33430,Volkswagen,18,18,18,13,0,0,0,0,0,0,0,0,67,제주,법인,-,2021


In [60]:
d

,Brand,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,total,지역,유형,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33427,Peugeot,2,0,14,27,0,0,0,0,0,0,0,0,43,제주,법인,-,2021
33428,Porsche,3,1,2,5,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
33429,Toyota,1,4,3,3,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
33430,Volkswagen,18,18,18,13,0,0,0,0,0,0,0,0,67,제주,법인,-,2021


In [61]:
# 연 월 브랜드 모델 연령 성별 지역 등록 대수

In [85]:
data = d.groupby(["Brand", "지역", "유형", "age", "year"]).sum(0).reset_index()
data.to_excel('clean_data.xlsx')
data.to_csv('clean_data.csv', index=False, encoding='utf-8-sig')

In [29]:
c = d.drop('total', axis=1).iloc[:,-1:]
c

,year
0,2016
1,2016
2,2016
3,2016
4,2016
...,...
33427,2021
33428,2021
33429,2021
33430,2021


In [20]:
c.T

,0,1,2,3,4,5,6,7,8,9,...,33422,33423,33424,33425,33426,33427,33428,33429,33430,33431
Brand,MINI,Volvo,Audi,BMW,Cadillac,Chrysler,Citroen,Fiat,Ford,Honda,...,Lexus,Lincoln,MINI,Maserati,Mercedes-Benz,Peugeot,Porsche,Toyota,Volkswagen,Volvo
1월,0,0,2,26,0,8,1,2,7,3,...,5,2,14,0,69,2,3,1,18,20
2월,0,0,4,44,2,6,4,5,1,1,...,4,3,17,0,67,0,1,4,18,26
3월,0,1,0,70,0,4,1,6,7,3,...,6,5,8,0,135,14,2,3,18,29
4월,0,0,0,94,0,2,2,2,3,8,...,2,3,3,1,166,27,5,3,13,23
5월,0,0,0,82,0,7,1,1,5,5,...,0,0,0,0,0,0,0,0,0,0
6월,0,0,0,81,1,9,1,1,5,7,...,0,0,0,0,0,0,0,0,0,0
7월,0,0,0,37,3,9,4,0,7,7,...,0,0,0,0,0,0,0,0,0,0
8월,0,0,0,57,0,9,2,0,4,0,...,0,0,0,0,0,0,0,0,0,0
9월,1,0,0,76,0,14,1,0,4,9,...,0,0,0,0,0,0,0,0,0,0


In [44]:
def select_menu(driver, area, area_id, sales_type_id):
    
    # 메뉴 초기화
    clear_menu(driver)
    time.sleep(0.5)
    
    # 지역 클릭
    driver.find_element_by_css_selector(f'input#{area_id}').click()
    time.sleep(0.5)
    
    # 상세 지역 클릭 : 전체가 클릭되도록 설정 : 이 메뉴가 안나오는 경우도 있으므로 예외처리
    try:
        driver.find_element_by_css_selector(f'input#all-{area}').click()
        time.sleep(0.5)
    except:
        pass
    
    # 구매유형 클릭
    driver.find_element_by_css_selector(f'input#{sales_type_id}').click()
    time.sleep(0.5)
    
    # 연령대 클릭
    try: 
        driver.find_element_by_css_selector(f'input#{age_id}').click()
        time.sleep(0.5)
    except:
        pass

In [65]:
# 법인만 따로 
# 6. 함수 실행
dfs = []

for year in range(2021, 2022): #2016, 2022        
    years = Select(driver.find_element_by_xpath("""//*[@id="searchYear"]"""))
    years.select_by_value(f'{year}')  
    for area, area_id in list(areas.items()):
        for sales_type, sales_type_id in list(sales_types.items())[-1:]: #법인은 오류가 나서 뺐습니다 ~.~
            print([area, sales_type, year], end=" ")

                    # 메뉴 선택
            select_menu(driver, area, area_id, sales_type_id)

                    # 검색버튼 클릭
            driver.find_element_by_css_selector('.searchBtn').click()
            time.sleep(0.5)

                    # 데이터 수집해서 데이터 프레임으로 만들기
            df = make_table_df(driver, area, sales_type, age)
            dfs.append(df)

                
# 수집된 데이터 프레임 병합
result_df = pd.concat(dfs)


# 출력
result_df.tail()

['서울', '법인', 2021] ['부산', '법인', 2021] ['대구', '법인', 2021] ['인천', '법인', 2021] ['광주', '법인', 2021] ['대전', '법인', 2021] ['울산', '법인', 2021] ['세종', '법인', 2021] ['경기', '법인', 2021] ['강원', '법인', 2021] ['충북', '법인', 2021] ['충남', '법인', 2021] ['전북', '법인', 2021] ['전남', '법인', 2021] ['경북', '법인', 2021] ['경남', '법인', 2021] ['제주', '법인', 2021] 

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,area,sales_type,age
15,Peugeot,2,0,14,27,0,0,0,0,0,0,0,0,43,제주,법인,60~69
16,Porsche,3,1,2,5,0,0,0,0,0,0,0,0,11,제주,법인,60~69
17,Toyota,1,4,3,3,0,0,0,0,0,0,0,0,11,제주,법인,60~69
18,Volkswagen,18,18,18,13,0,0,0,0,0,0,0,0,67,제주,법인,60~69
19,Volvo,20,26,29,23,0,0,0,0,0,0,0,0,98,제주,법인,60~69


In [68]:
result16 = pd.read_excel('result_16.xlsx')
result17 = pd.read_excel('result_17.xlsx')
result18 = pd.read_excel('result_18.xlsx')
result19 = pd.read_excel('result_19.xlsx')
result20 = pd.read_excel('result_20.xlsx')
result21 = pd.read_excel('result_21.xlsx')

In [70]:
type_4 = pd.concat([result16,result17,result18,result19,result20,result21])

In [71]:
type_4.to_csv('sales_type4.csv', index=False, encoding='utf-8-sig')

In [73]:
data = pd.read_csv('kaida.csv')
data

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,area,sales_type,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31424,Volvo,0,3,1,0,0,0,0,0,0,0,0,0,4,제주,개인-여자,50~59,2021
31425,Chrysler,2,0,0,0,0,0,0,0,0,0,0,0,2,제주,개인-여자,60~69,2021
31426,Mercedes-Benz,1,0,1,3,0,0,0,0,0,0,0,0,5,제주,개인-여자,60~69,2021
31427,Porsche,1,0,0,1,0,0,0,0,0,0,0,0,2,제주,개인-여자,60~69,2021


In [74]:
final = pd.concat([data, type_4])

In [75]:
final

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,area,sales_type,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Peugeot,2,0,14,27,0,0,0,0,0,0,0,0,43,제주,법인,-,2021
299,Porsche,3,1,2,5,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
300,Toyota,1,4,3,3,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
301,Volkswagen,18,18,18,13,0,0,0,0,0,0,0,0,67,제주,법인,-,2021


In [76]:
final.to_csv('kaida.csv', index=False, encoding='utf-8-sig')

In [77]:
final.columns = ["Brand", "1월", "2월", "3월", "4월", "5월", "6월", "7월", "8월", "9월", "10월", "11월", "12월", "total", "지역", "유형", "age", "year"]

In [78]:
final

,Brand,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,total,지역,유형,age,year
0,MINI,0,0,0,0,0,0,0,0,1,0,0,0,1,서울,개인,~19,2016
1,Volvo,0,0,1,0,0,0,0,0,0,0,0,0,1,서울,개인,~19,2016
2,Audi,2,4,0,0,0,0,0,0,0,0,0,0,6,서울,개인,20~29,2016
3,BMW,26,44,70,94,82,81,37,57,76,70,60,98,795,서울,개인,20~29,2016
4,Cadillac,0,2,0,0,0,1,3,0,0,2,2,0,10,서울,개인,20~29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Peugeot,2,0,14,27,0,0,0,0,0,0,0,0,43,제주,법인,-,2021
299,Porsche,3,1,2,5,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
300,Toyota,1,4,3,3,0,0,0,0,0,0,0,0,11,제주,법인,-,2021
301,Volkswagen,18,18,18,13,0,0,0,0,0,0,0,0,67,제주,법인,-,2021


In [79]:
final.to_csv('kaida_korean.csv', index=False, encoding='utf-8-sig')